# Alachua County restaurant inspection analysis
_This will take restaurant inspection data by the state of Florida and format it in a more reader-friendly way for publication in print and online. We'll filter for the most egregious current violations at restaurants in Alachua County._

After importing Pandas,etc., this reads in the state summary report year-to-date for District 5, which includes Alachua County, and adds an exception in case the file is not found (output probably needs to be set as variable so it can be written into the output file). The raw file has no headers and 82 columns. So this removes all but five columns and adds headers for those. Finally, it displays the first five rows of values.

In [1]:
import pandas as pd
import datetime
import numpy as np
from urllib.request import urlopen
from urllib.error import HTTPError
from bs4 import BeautifulSoup

In [2]:
try:
    insp = pd.read_csv("ftp://dbprftp.state.fl.us/pub/llweb/5fdinspi.csv", 
                               usecols=[2,14,18,80,81])
    
except IOError:
    print("The file is not accessible.")
insp.columns = ["CountyName", "InspectDate", "NumHighVio", "LicenseID", "VisitID"]

insp.head() ## this can go away later

,CountyName,InspectDate,NumHighVio,LicenseID,VisitID
0,Alachua,11/27/2017,0.0,3713828,6267656
1,Alachua,01/11/2018,0.0,3713828,6432746
2,Alachua,11/27/2017,0.0,3713765,6267651
3,Alachua,12/13/2017,0.0,3713820,6267655
4,Alachua,03/28/2018,0.0,5399007,6510609


In [3]:
alachua = insp[insp.CountyName == 'Alachua']
alachua.info() ## this can go away later, but shows total number of rows

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1528 entries, 0 to 1527
Data columns (total 5 columns):
CountyName     1528 non-null object
InspectDate    1528 non-null object
NumHighVio     1528 non-null float64
LicenseID      1528 non-null int64
VisitID        1528 non-null int64
dtypes: float64(1), int64(2), object(2)
memory usage: 71.6+ KB


In [4]:
alachua = alachua[alachua.NumHighVio > 0]
alachua.info() ## this can go away later, but shows how many rows filtered out

<class 'pandas.core.frame.DataFrame'>
Int64Index: 767 entries, 5 to 1514
Data columns (total 5 columns):
CountyName     767 non-null object
InspectDate    767 non-null object
NumHighVio     767 non-null float64
LicenseID      767 non-null int64
VisitID        767 non-null int64
dtypes: float64(1), int64(2), object(2)
memory usage: 36.0+ KB


In [5]:
alachua['InspectDate'] = pd.to_datetime(alachua['InspectDate']) ## changes date string to date object
alachua = alachua.sort_values('InspectDate', ascending=False) ## sort DateFrame, most recent first
alachua.info() ## this can go away later

<class 'pandas.core.frame.DataFrame'>
Int64Index: 767 entries, 1342 to 842
Data columns (total 5 columns):
CountyName     767 non-null object
InspectDate    767 non-null datetime64[ns]
NumHighVio     767 non-null float64
LicenseID      767 non-null int64
VisitID        767 non-null int64
dtypes: datetime64[ns](1), float64(1), int64(2), object(1)
memory usage: 36.0+ KB


__Goal with this next is to select date range prior to 'today', but this is hard coded now.

In [6]:
today = pd.to_datetime('today')
startDay = datetime.date.today() - datetime.timedelta(days=30)
## want to get user input for timedelta 
alachua = alachua[(alachua['InspectDate'] > startDay) & (alachua['InspectDate'] < today)]

alachua.info()
alachua.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 44 entries, 1342 to 126
Data columns (total 5 columns):
CountyName     44 non-null object
InspectDate    44 non-null datetime64[ns]
NumHighVio     44 non-null float64
LicenseID      44 non-null int64
VisitID        44 non-null int64
dtypes: datetime64[ns](1), float64(1), int64(2), object(1)
memory usage: 2.1+ KB


,CountyName,InspectDate,NumHighVio,LicenseID,VisitID
1342,Alachua,2018-05-04,1.0,6312191,6539001
869,Alachua,2018-05-03,1.0,6040839,6292423
680,Alachua,2018-05-03,1.0,6475603,6444510
611,Alachua,2018-05-03,2.0,6370423,6463635
89,Alachua,2018-05-02,1.0,2129170,6535653


__Loop takes the LicenseID and VisitID, passes it to the url for the detailed reports:__

In [8]:
for index, rows in alachua.iterrows():
    visitID = rows['VisitID']
    licID = rows['LicenseID']
    urls = "https://www.myfloridalicense.com/inspectionDetail.asp?InspVisitID= %s &licid= %s" % (visitID, licID)
    urls = urls.replace(' ', '')
    print(urls) # this can go away

https://www.myfloridalicense.com/inspectionDetail.asp?InspVisitID=6539001&licid=6312191
https://www.myfloridalicense.com/inspectionDetail.asp?InspVisitID=6292423&licid=6040839
https://www.myfloridalicense.com/inspectionDetail.asp?InspVisitID=6444510&licid=6475603
https://www.myfloridalicense.com/inspectionDetail.asp?InspVisitID=6463635&licid=6370423
https://www.myfloridalicense.com/inspectionDetail.asp?InspVisitID=6535653&licid=2129170
https://www.myfloridalicense.com/inspectionDetail.asp?InspVisitID=6535804&licid=3296677
https://www.myfloridalicense.com/inspectionDetail.asp?InspVisitID=6535766&licid=5430232
https://www.myfloridalicense.com/inspectionDetail.asp?InspVisitID=6451801&licid=5858850
https://www.myfloridalicense.com/inspectionDetail.asp?InspVisitID=6247794&licid=2129170
https://www.myfloridalicense.com/inspectionDetail.asp?InspVisitID=6433038&licid=6621480
https://www.myfloridalicense.com/inspectionDetail.asp?InspVisitID=6451852&licid=6665274
https://www.myfloridalicense.com

__Now it's time to use those urls to access the detailed reports with inspector comments, and scrape those.__

In [10]:
len(urls) # this can go away

87

In [13]:
# here's one approach

def getText(urls):
    html = urlopen(urls)
    inspectReport = BeautifulSoup(html.read(), 'html.parser')
    inspectText = inspectReport.findAll('font', {'face':'verdana'})
    
    text = getText('https://www.myfloridalicense.com/inspectionDetail.asp?InspVisitID=6509808&licid=5399007')

for text in inspectText:
        print(text.get_text())

NameError: name 'inspectText' is not defined

None
